In [7]:
import pandas as pd
from ydata_profiling import ProfileReport
from tqdm import tqdm

In [8]:
df = pd.read_pickle("../tmp/product_tables.pickle")

In [7]:
for col in tqdm(df.columns):

    df = df.explode(col)
    print(col)
    print(df.shape)

  1%|          | 1/114 [00:00<00:57,  1.95it/s]

PRODUCTCODE
(21298, 114)


  2%|▏         | 2/114 [00:00<00:46,  2.39it/s]

pdt_capacity_TOURGRADECODE
(21298, 114)


  3%|▎         | 3/114 [00:01<00:47,  2.31it/s]

...

 17%|█▋        | 19/114 [00:12<02:38,  1.66s/it]

pdt_incexcl_VIDESTINATIONCITY
(481426, 114)


 18%|█▊        | 20/114 [00:17<04:33,  2.91s/it]

pdt_inclexcl_TYPE
(920652, 114)


: 

In [ ]:
df

In [ ]:
profile = ProfileReport(df, title="Profiling Report")

In [7]:
profile.to_file("../tmp/output.html")

In [ ]:
def clean_dataframe(df):

    df.fillna("", inplace=True)
    
    # For numerical columns, you might want to fill with mean/median
    for col in df.select_dtypes(include=[np.number]).columns:
        df[col].fillna(df[col].mean(), inplace=True)
        
    # Convert categorical columns with fill missing values and encode
    categorical_cols = df.select_dtypes(include=[object]).columns
    for col in categorical_cols:
        df[col].fillna('unknown', inplace=True)
        if df[col].nunique() < 10:  # Using one-hot encoding for columns with fewer unique values
            df = pd.get_dummies(df, columns=[col], drop_first=True)
        else:  # Using label encoding for columns with many unique values
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
    
    # Handling columns with lists
    list_cols = ['list_col1', 'list_col2']  # Replace with your actual list columns
    for col in list_cols:
        df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [])
        
        # Example: flatten list columns by converting list of numbers to mean value
        if df[col].apply(lambda x: all(isinstance(i, (int, float)) for i in x)).all():
            df[col] = df[col].apply(lambda x: np.mean(x) if len(x) > 0 else 0)
        # Example: convert list of lists to string or other appropriate representation
        elif df[col].apply(lambda x: all(isinstance(i, list) for i in x)).all():
            df[col] = df[col].apply(lambda x: ' '.join(map(str, x)) if len(x) > 0 else '')
    
    # Geolocation - assuming they are in format (lat, long)
    if 'geolocation' in df.columns:
        df[['latitude', 'longitude']] = df['geolocation'].apply(pd.Series)
        df.drop('geolocation', axis=1, inplace=True)
        
    return df

# Apply the cleaning function
cleaned_df = clean_dataframe(df)

# Checking correlation
correlation_matrix = cleaned_df.corr()

# Displaying high correlations
high_correlation_pairs = correlation_matrix.abs().unstack().sort_values(kind="quicksort", ascending=False)
high_correlation_pairs = high_correlation_pairs[high_correlation_pairs != 1]  # Exclude self-correlations

print(high_correlation_pairs.head(10))  # Print top 10 highest correlations